In [4]:
# Standard libraries
import pandas as pd
import numpy as np
import os

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Display settings
# Display more rows and columns
pd.set_option("display.max_rows", 400)
pd.set_option("display.max_columns", 600)
pd.set_option("display.width", 1000)

# Optional: Plot styling
sns.set(style="whitegrid")

In [5]:
file_path = "../data/raw/data_mutations.txt"

df = pd.read_csv(file_path, sep="\t", comment="#", low_memory=False)

In [7]:
dup_count = df.duplicated().sum()
print(f'Number of fully duplicated rows: {dup_count}')

Number of fully duplicated rows: 0


In [8]:
df.head()

,Hugo_Symbol,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_Position,End_Position,Strand,Consequence,Variant_Classification,Variant_Type,Reference_Allele,Tumor_Seq_Allele1,Tumor_Seq_Allele2,dbSNP_RS,dbSNP_Val_Status,Tumor_Sample_Barcode,Matched_Norm_Sample_Barcode,Match_Norm_Seq_Allele1,Match_Norm_Seq_Allele2,Tumor_Validation_Allele1,Tumor_Validation_Allele2,Match_Norm_Validation_Allele1,Match_Norm_Validation_Allele2,Verification_Status,Validation_Status,Mutation_Status,Sequencing_Phase,Sequence_Source,Validation_Method,Score,BAM_File,Sequencer,t_ref_count,t_alt_count,n_ref_count,n_alt_count,HGVSc,HGVSp,HGVSp_Short,Transcript_ID,RefSeq,Protein_position,Codons,Exon_Number,1000G_AF,1000G_AFR_AF,1000G_AMR_AF,1000G_EAS_AF,1000G_EUR_AF,1000G_SAS_AF,APPRIS,Allele,Amino_acids,BIOTYPE,CANONICAL,CCDS,CDS_position,CLIN_SIG,CONTEXT,COSMIC,DISTANCE,DOMAINS,ENSP,ESP_AA_AF,ESP_EA_AF,EXON,Existing_variation,FLAGS,Feature,Feature_type,GDC_FILTER,GENE_PHENO,Gene,HGNC_ID,HGVS_OFFSET,HIGH_INF_POS,IMPACT,INTRON,MANE,MAX_AF,MAX_AF_POPS,MOTIF_NAME,MOTIF_POS,MOTIF_SCORE_CHANGE,One_Consequence,PHENO,PICK,PUBMED,PolyPhen,RNA_Support,RNA_alt_count,RNA_depth,RNA_ref_count,SIFT,SOMATIC,SWISSPROT,SYMBOL,SYMBOL_SOURCE,TRANSCRIPTION_FACTORS,TRANSCRIPT_STRAND,TREMBL,TSL,UNIPARC,UNIPROT_ISOFORM,VARIANT_CLASS,all_effects,cDNA_position,callers,case_id,genomic_location_explanation,gnomAD_AF,gnomAD_AFR_AF,gnomAD_AMR_AF,gnomAD_ASJ_AF,gnomAD_EAS_AF,gnomAD_FIN_AF,gnomAD_NFE_AF,gnomAD_OTH_AF,gnomAD_SAS_AF,gnomAD_non_cancer_AF,gnomAD_non_cancer_AFR_AF,gnomAD_non_cancer_AMI_AF,gnomAD_non_cancer_AMR_AF,gnomAD_non_cancer_ASJ_AF,gnomAD_non_cancer_EAS_AF,gnomAD_non_cancer_FIN_AF,gnomAD_non_cancer_MAX_AF_POPS_adj,gnomAD_non_cancer_MAX_AF_adj,gnomAD_non_cancer_MID_AF,gnomAD_non_cancer_NFE_AF,gnomAD_non_cancer_OTH_AF,gnomAD_non_cancer_SAS_AF,hotspot,miRNA,n_depth,normal_bam_uuid,t_depth,tumor_bam_uuid,Annotation_Status
0,MTCL1,23255,BI,GRCh38,18,8806948,8806948,+,missense_variant,Missense_Mutation,SNP,G,G,A,rs199934266,NaN,TCGA-AB-2881-03A,TCGA-AB-2881-11A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Somatic,NaN,NaN,NaN,NaN,NaN,NaN,26,6,NaN,NaN,ENST00000306329.15:c.3449G>A,p.Arg1150His,p.R1150H,ENST00000306329,NM_001378206.1,1150,cGt/cAt,9/14,NaN,NaN,NaN,NaN,NaN,NaN,P3,A,R/H,protein_coding,YES,NaN,3449/5718,NaN,CTTCCGTGCGG,COSM1318381,NaN,Coiled-coils_(Ncoils):Coil;PANTHER:PTHR15742;P...,ENSP00000305027,0.000000,0.000116,9/14,rs199934266,NaN,ENST00000306329,Transcript,NaN,NaN,ENSG00000168502,HGNC:29121,NaN,NaN,MODERATE,NaN,NaN,0.000124,gnomAD_NFE,NaN,NaN,NaN,missense_variant,NaN,1.0,NaN,benign(0.045),Unknown,NaN,NaN,NaN,tolerated(0.16),NaN,Q9Y4B5.139,MTCL1,HGNC,NaN,1,NaN,5.0,UPI0001AE65C5,Q9Y4B5-1,SNV,"MTCL1,missense_variant,p.R831H,ENST00000359865...",3449/5718,mutect2;varscan2,fff35c80-88cd-4923-80c1-0273ba5bed0f,NaN,0.000072,0.000000,0.000029,0.000000,0.000000,0.000095,0.000124,0.000000,0.000033,0.000047,0.000000,0.0,0.000000,0.0,0.000000,0.000000,nfe,0.000108,0.0,0.000108,0.0,0.000000,0,NaN,39,ae212e50-9fb6-47c6-b334-836962553043,32,d1373e1b-d60d-405d-aeb4-ae045f03e331,SUCCESS
1,HSH2D,84941,BI,GRCh38,19,16157257,16157257,+,inframe_insertion,In_Frame_Ins,INS,G,G,CTCC,novel,NaN,TCGA-AB-2881-03A,TCGA-AB-2881-11A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Somatic,NaN,NaN,NaN,NaN,NaN,NaN,89,9,NaN,NaN,ENST00000613986.4:c.522delinsCTCC,p.Ser175dup,p.S175dup,ENST00000613986,NM_001352265.2,175,ccG/ccCTCC,6/6,NaN,NaN,NaN,NaN,NaN,NaN,P1,CTCC,P/PS,protein_coding,NaN,CCDS74304.1,522/1059,NaN,AAGCCGTCAGC,NaN,NaN,PANTHER:PTHR14388;PANTHER:PTHR14388:SF3;MobiDB...,ENSP00000483354,NaN,NaN,6/6,NaN,NaN,ENST00000613986,Transcript,NaN,NaN,ENSG00000196684,HGNC:24920,NaN,NaN,MODERATE,NaN,NM_001382417.1,NaN,NaN,NaN,NaN,NaN,inframe_insertion,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,Q96JZ2.142,HSH2D,HGNC,NaN,1,NaN,2.0,UPI0000073F82,Q96JZ2-1,indel,"HSH2D,inframe_insertion,p.S175dup,ENST00000616...",639/1957,pindel;varscan2*,fff35c80-88cd-4923-80c1-0273ba5bed0f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

### Duplicate Check

I confirmed in the previous notebook that there are **no duplicate rows** in the raw dataset:
`df.duplicated().sum() == 0`

This is normal for mutation datasets, where each row represents a unique mutation event.

Because of this, the cleaning process starts with missing values, columns, and consistency.


In [10]:
missing_pct = df.isna().mean().sort_values(ascending=False)
missing_pct.head(20)

genomic_location_explanation     1.0
TRANSCRIPTION_FACTORS            1.0
n_ref_count                      1.0
Sequencer                        1.0
BAM_File                         1.0
Score                            1.0
Validation_Method                1.0
Sequence_Source                  1.0
Sequencing_Phase                 1.0
Validation_Status                1.0
Verification_Status              1.0
Match_Norm_Validation_Allele2    1.0
Match_Norm_Validation_Allele1    1.0
Tumor_Validation_Allele2         1.0
Tumor_Validation_Allele1         1.0
Match_Norm_Seq_Allele2           1.0
Match_Norm_Seq_Allele1           1.0
MOTIF_NAME                       1.0
MOTIF_POS                        1.0
RNA_ref_count                    1.0
dtype: float64

### Columns with 100% Missing Values

During missingness analysis, I found several columns with a missingness score of **1.0**, 
meaning they contain no usable data at all.

These fields are optional metadata fields in the MAF (Mutation Annotation Format), 
but in this dataset they were not populated by the upstream mutation-calling pipeline.

Examples include:
- Sequencing metadata (Sequencer, BAM_File)
- Validation fields (Validation_Method, Validation_Status)
- Normal-sample allele fields
- Motif or transcription-factor annotations
- RNA read count fields

Since these columns provide no information, they are removed from the cleaned dataset.

This step simplifies the dataset and ensures future analysis and visualizations only use meaningful and populated features.

In [11]:
empty_cols = missing_pct[missing_pct == 1.0].index.tolist()
print("Columns with 100% missing:", empty_cols)

df_clean = df.drop(columns=empty_cols)


Columns with 100% missing: ['genomic_location_explanation', 'TRANSCRIPTION_FACTORS', 'n_ref_count', 'Sequencer', 'BAM_File', 'Score', 'Validation_Method', 'Sequence_Source', 'Sequencing_Phase', 'Validation_Status', 'Verification_Status', 'Match_Norm_Validation_Allele2', 'Match_Norm_Validation_Allele1', 'Tumor_Validation_Allele2', 'Tumor_Validation_Allele1', 'Match_Norm_Seq_Allele2', 'Match_Norm_Seq_Allele1', 'MOTIF_NAME', 'MOTIF_POS', 'RNA_ref_count', 'RNA_depth', 'RNA_alt_count', 'MOTIF_SCORE_CHANGE', 'miRNA', 'n_alt_count', 'HIGH_INF_POS', 'dbSNP_Val_Status']
